<a href="https://colab.research.google.com/github/jamunozlab/introductory_mechanics_fall_2023/blob/main/projects/Phys_2320_project_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Backstory

In the past few weeks, Jorginho has been disciplined enough and following the training program the profesionals at your company Physics Physical LLC have designed for him. He is close to attaining the times your company requested of him: photographic evidence is located [here](https://github.com/jamunozlab/introductory_mechanics_fall_2022/blob/main/projects/Ph_2320_P3_data.ipynb) and is summarized below.



```
# Date         # Starting speed   # Predicted time    # Actual time
# 9/10/23        6.1 mph            53:45 min:sec       53:23 min:sec
# 9/17/23        6.2 mph            52:59 min:sec       53:01 min:sec
# 9/24/23        6.3 mph            52:16 min:sec       52:33 min:sec
# 10/1/23        6.4 mph            51:35 min:sec       52:04 min:sec
# 10/8/23        6.5 mph            50:58 min:sec       51:21 min:sec
# 10/15/23       6.6 mph            50:23 min:sec
# 10/22/23       6.7 mph            49:51 min:sec
# 10/29/23       <-- Race day (real deal)
```

Nevertheless, Jorginho is complaining that even though he followed the running program closely on 9/24, 10/1, and 10/8, his times are not quite the same you had predicted. You suddendly remember that wikipedia article about [reductionism](https://en.wikipedia.org/wiki/Reductionism) that says: ``reductionist thinking and methods form the basis for many of the well-developed topics of modern science, including much of physics, chemistry and molecular biology. Classical mechanics in particular is seen as a reductionist framework.'' Although you are not sure about ontological reductionism (a belief that the whole of reality consists of a minimal number of parts), methodological reductionism (the scientific attempt to provide an explanation in terms of ever-smaller entities) has proved its worth in science and technology. You have seen how GPS data, collected every second, can be used in the same way as the $\vec{x}(t)$ kinematic equation. In particular, you are suspicious of the assumption that the treadmill increases its velocity infinitely fast. You decide to do a dynamics simulation to improve your predictions.

## Goals of the project

In this project, you will find the finite value of the acceleration of the tread mill when the user increases the speed.

Evaluate the cells below as necessary

In [1]:
# Some packages to make our lives easy and print pretty plots
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

In [2]:
# This function takes a running routine, which consists of a list of positions
# and a list of velocities for the treadmill corresponding to the positions in the list
# It also takes a distance and a velocity
# It returns 'True' if the velocity is correct for that distance according to the program
# It returns 'False' otherwise
# In every case, it also returns the velocity that the treadmill should have for that distance

def check_correct_speed(distance, velocity, position_list, velocity_list):
  for i in range(len(position_list)):
    # Find the index corresponding to the current position
    if distance >= position_list[i] and distance < position_list[i+1]:
      idx = i
      break

  # So that we know at which speed Jorginho should be running
  # And compare to Jorginho's actual speed
  correct = False if velocity - velocity_list[idx] < 0 else True

  # Return True if Jorginho is running at the correct speed, False otherwhise
  return correct, velocity_list[idx]

In [3]:
#####################
## Running Routine ##
#####################

# This is a running program like the one you built in Project 1
position_list = [0, 1/3, 2/3, 3/3, 4/3, 5/3, 6/3, 7/3, 8/3, 9/3, 10/3, 11/3, 12/3, 13/3, 14/3, 15/3, 16/3, 17/3, 18/3, 19/3]
velocity_list = [   6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3,  7.4,  7.5, 7.6,   7.7,  7.8,  7.9,  8.0,  8.0,  8.0,  8.0]

# For convenience, we are moving to SI (1 mile is 1609 meters and 1 hour is 3600 seconds)
position_list_m = [position * (1609.0) for position in position_list] # Distance in meters
velocity_list_m = [velocity * (1609.0/3600.0) for velocity in velocity_list] # Velocity in meters/second

In [ ]:
#########################
## Dynamics Simulation ##
#########################

# For this situation, the acceleration can only be positive since Jorginho is not running backwards
# The acceleration should be less than 1 m/s^2, which is reasonable for a long-distance running
# You can change it below, the acceleration is only 'on' when the velocity should be increased
acceleration_when_on = 0.0166 # m/s^2

# Make sure acceleration is within range
assert acceleration_when_on > 0, "Problem with acceleration: When Jorginho accelerates, the acceleration is positive"
assert acceleration_when_on <= 1, "Problem with acceleration: The maximum acceleratino of Jorginho is 1 m/s^2"

# delta_time tells the code how often to check/update variables
# a small delta_time gives more accurate results but will take longer to compute
# initial position, velocity, and time initialized
delta_time = 0.10
distance = 0.0
velocity = 0.0
time = 0

# This is where we are going to save our dynamics data, simple lists
actual_time_list = []
actual_distance_list = []
actual_velocity_list = []
actual_acceleration_list = []


verbose = True
print_every = int(1/delta_time) # print info periodically
count = 0
while True:
  # Below you have your friend the kinematic equation for position vs time
  # The distance is updated every time this code runs (so every second)
  distance = distance + velocity * delta_time + (1/2) * acceleration_when_on * delta_time**2

  # This will check whether the velocity of the treadmill corresponds to the velocity it should have
  # according to your running program
  # It also gives you what that velocity should be
  check_correct, velocity_should_be = check_correct_speed(distance, velocity, position_list_m, velocity_list_m)

  # If the velocity is correct, then nothing happens, the velocity is maintained
  # If the velocity is incorrect, then it turns on the acceleration, so the velocity increases
  acceleration = 0 if check_correct else acceleration_when_on

  # Below you have your friend the kinematic equation for velocity vs time
  # Although if put it in an if/else statement so that I can give it the velocity it should be
  # This helps avoid the accumulation of small errors, so it is a bit of a trick
  velocity = velocity + acceleration * delta_time if not check_correct else velocity_should_be

  # The line below will check if the distance has reached 10,000 meters (a 10K)
  # If yes, break the while loop, Jorginho is done; if no, just continue for 1 more second
  if distance > 10000:
    print('| time (seconds):', int(time), '| distance (meters):', int(distance), '| velocity (m/s):', velocity, '| acceleration (m/s^2):', acceleration, '|')
    break

  # This prints out time, distance, velocity, and acceleration for each second
  if 0 == count % print_every and verbose is True:
    print('| time (seconds):', int(time), '| distance (meters):', int(distance), '| velocity (m/s):', velocity, '| acceleration (m/s^2):', acceleration, '|')

  # This updates the time so that in the next iteration the time has advanced by 1 second
  time = time + delta_time
  count = count + 1

  # This is just so that we can put the data in an easy to manipulate format
  actual_time_list.append(time)
  actual_distance_list.append(distance)
  actual_velocity_list.append(velocity)
  actual_acceleration_list.append(acceleration)

# Gives last bit of information, the time it took Jorginho to actually run the 10k given you program
# and the limitations of the treadmill
print()
print('Total time was:', time, 'seconds')
print('Total time was:', int(time/60), 'minutes and', np.round((time/60 - int(time/60)) * 60), 'seconds')

In [5]:
# Prettyfy
df_dict = {}
df_dict['distance'] = pd.Series(actual_distance_list, index=actual_time_list)
df_dict['velocity'] = pd.Series(actual_velocity_list, index=actual_time_list)
df_dict['acceleration'] = pd.Series(actual_acceleration_list, index=actual_time_list)
df = pd.DataFrame(df_dict)

In [ ]:
# Plot distance vs time
df['distance'].plot()

In [ ]:
# Plot velocity vs time
df['velocity'].plot()

In [ ]:
# Plot acceleration vs time
df['acceleration'].plot()

In [ ]:
# Plot velocity vs position
df.plot(x='distance', y='velocity')

# Problem 3.1

The running program for Jorginho for 10/8/2023 is the cell above marked as:

```
#####################
## Running Routine ##
#####################
```
and the dynamics simulation is in the cell marked above as:
```
#########################
## Dynamics Simulation ##
#########################
```

3.1.1 Change the value of the variable `acceleration_when_on' and run the dynamics simulation. Also evaluate the cells that produce the plots. What value of the acceleration allows you to get a time as close as possible to Jorginho's actual time of 51:21? Write down your answer below.











In [10]:
## 3.1.1 What is the value of acceleration_on that produces the closes time to 51 minutes and 21 seconds?
## The starting speed is 6.5 miles per hour
##
## Answers:

3.1.2. Modify the running program so that it simulates Jorginho run for 10/1/2023, which he started at a speed of 6.4 mph.

Change the value of the variable `acceleration_when_on' and run the dynamics simulation. Also evaluate the cells that produce the plots. What value of the acceleration allows you to get a time as close as possible to Jorginho's actual time of 52:04? Write down your answer below.

In [11]:
## 3.1.2 What is the value of acceleration_on that produces the closes time to 52 minutes and 4 seconds?
## The starting speed is 6.4 miles per hour
##
## Answers:

## Problem 3.2

3.2.1. Analyze the plots produced using your simulation data. What are the assumptions in your Project 1 code that are not valid or completely correct?

3.2.2. Have they been corrected or ameliorated in this version of the code?

3.2.3 What can you do to improve your predictions further?

In [12]:
## 3.2.1 What assumptions from Project 1 are questionable?
##
## Answers:

In [13]:
## 3.2.2 Did you correct them or ameliorated them in this version of the code?
##
## Answers:

In [14]:
## 3.2.3 Provide ideas to improve the simulation further
##
## Answers:

## Problem 3.3

The good thing about running faster is that you finish sooner. Look at the data generated by your simulation for Jorginho's 10/1/2023 run.

3.3.1 Divide the total distance in 2 halves.  What was his time for the first half (distance-wise) and for the second one?

3.3.2 Now divide the total time in 2 halves. What distance did he run during the first half (time-wise) and during the second one?

In [15]:
## 3.3.1 What was Jorginho's time for the first half and second half of the run, distance-wise?
##
## Answers:

In [16]:
## 3.3.2 What was Jorginho's distance for the first half and second half of the run, time-wise?
##
## Answers: